<a href="https://colab.research.google.com/github/yudumpacin/NLPStudyNotes/blob/main/Train_Turkish_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [68]:
sample = "Dr. Aydın, cuma günü saat 3'te İst. Ünv. Hastanesi'nde bir konferans verecek."
tokens = nltk.word_tokenize(sample)

print(tokens)


['Dr.', 'Aydın', ',', 'cuma', 'günü', 'saat', "3'te", 'İst', '.', 'Ünv', '.', "Hastanesi'nde", 'bir', 'konferans', 'verecek', '.']


In [69]:
import spacy

In [ ]:
pip install https://huggingface.co/turkish-nlp-suite/tr_core_news_md/resolve/main/tr_core_news_md-any-py3-none-any.whl

In [72]:
nlp = spacy.load("tr_core_news_md")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'tr_core_news_md' (3.4.2) was trained with spaCy v3.4.2 and may not be 100% compatible with the current version (3.6.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [74]:
doc = nlp(sample)
for token in doc:
  print(token, token.pos_)

Dr. NOUN
Aydın VERB
, PUNCT
cuma NOUN
günü NOUN
saat NOUN
3'te NUM
İst. NOUN
Ünv NOUN
. PUNCT
Hastanesi'nde NOUN
bir DET
konferans NOUN
verecek VERB
. PUNCT


In [4]:
import pandas as pd
import gensim
from gensim.models import Word2Vec,KeyedVectors

# Turkish Training Data

Data source is A Benchmark Data for Turkish Text Categorization on Kaggle https://www.kaggle.com/datasets/savasy/ttc4900


In [23]:
!pip install -q kaggle

In [24]:
!mkdir -p ~/.kaggle

In [26]:
!cp kaggle.json ~/.kaggle

In [27]:
!chmod 600 /root/.kaggle/kaggle.json

In [28]:
!kaggle datasets download -d savasy/ttc4900

  0% 0.00/3.34M [00:00<?, ?B/s]
100% 3.34M/3.34M [00:00<00:00, 230MB/s]


In [ ]:
!unzip /content/ttc4900

Archive:  /content/ttc4900.zip
replace 7allV03.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [31]:
df = pd.read_csv("/content/7allV03.csv")

In [32]:
df.head()

,category,text
0,siyaset,3 milyon ile ön seçim vaadi mhp nin 10 olağan...
1,siyaset,mesut_yılmaz yüce_divan da ceza alabilirdi pr...
2,siyaset,disko lar kaldırılıyor başbakan_yardımcısı ar...
3,siyaset,sarıgül anayasa_mahkemesi ne gidiyor mustafa_...
4,siyaset,erdoğan idamın bir haklılık sebebi var demek ...


In [36]:
df.shape

(4900, 2)

# Preprocess the Text


In [61]:
def preprocess(text_data):
  text_data = re.sub("[^a-zA-ZğüşöçıİĞÜŞÖÇ0-9.,;:!?'\" ]"," ",text_data)
  return text_data

In [62]:
df['text_cleaned'] = df["text"].apply(preprocess)

In [63]:
text = df['text_cleaned'].values

In [64]:
text[0]

' 3 milyon ile ön seçim vaadi mhp nin 10 olağan büyük kurultayı nda konuşan genel başkan adayı koray aydın seçimlerden önce partinin üye sayısının 3 milyona ulaştırılması hedefini koyarak ön seçim uygulaması vaadinde bulundu mhp nin 10 olağan büyük kurultayı nda konuşan genel başkan adayı koray aydın seçimlerden önce partinin üye sayısının 3 milyona ulaştırılması hedefini koyarak ön seçim uygulaması vaadinde bulundu genel başkan adayı koray aydın kürsüye beklenirken yapılan tezahüratlar ve ıslıklamalar üzerine divan başkanı tuğrul türkeş mhp nin genel başkanlığı da genel başkan adaylığı da saygıdeğer işlerdir bu salondaki herkes ciddiye almak zorundadır dedi ve taşkınlıklara izin verilmeyeceğini salonda sükunet sağlanmadan konuşmaların başlamayacağını vurguladı türkeş devlet bahçeli nin kurultay açılışında konuştuğu için adaylık nedeniyle ikinci bir konuşma yapmayacağını açıkladı konuşmasında kurultayın mhp nin tek başına iktidarına vesile olmasını dileyen aydın ak parti nin mhp yi ele

In [80]:
len(text)

4900

In [75]:
tokenized_nltk = [nltk.word_tokenize(row) for row in text]

In [76]:
tokenized_nltk[0][0:5]

['3', 'milyon', 'ile', 'ön', 'seçim']

In [ ]:
tokenized_spacy = []
for sentence in text:
  doc = nlp(sentence)
  token = [token for token in doc ]
  tokenized_spacy.append(token)

# Model

In [83]:
model_with_nltk_token = Word2Vec(tokenized_nltk, min_count=1, vector_size=32)

In [ ]:
#model_with_spacy_tr_token = Word2Vec(tokenized_spacy, min_count=1, vector_size=32)

# Test

In [94]:
model_with_nltk_token.wv.most_similar('sabah')

[('akşam', 0.9699873328208923),
 ('günkü', 0.9698092937469482),
 ('sahnesinde', 0.9623849987983704),
 ('salon', 0.9562276601791382),
 ('gecesi', 0.9554785490036011),
 ('duruşma', 0.9551609754562378),
 ('tutuklu', 0.9536200165748596),
 ('sinemalarda', 0.9534684419631958),
 ('boeing', 0.9531444311141968),
 ('saldırıda', 0.9507744312286377)]

In [95]:
model_with_nltk_token.wv.most_similar('izmir')

[('tunceli', 0.9489995241165161),
 ('eskişehir', 0.9472288489341736),
 ('mersin', 0.9457154870033264),
 ('balıkesir', 0.942883312702179),
 ('alaattin', 0.9397173523902893),
 ('pisti', 0.9376327395439148),
 ('denizli', 0.9352000951766968),
 ('belediyesi', 0.9314376711845398),
 ('ipekçi', 0.9295763969421387),
 ('bitlis', 0.9264671206474304)]

In [88]:
model_with_nltk_token.wv.most_similar('kadın')

[('müzik', 0.9381021857261658),
 ('belgesel', 0.9225330352783203),
 ('blog', 0.9154289364814758),
 ('çocuk', 0.9153070449829102),
 ('seviye', 0.914525032043457),
 ('gruplardan', 0.914469838142395),
 ('erkek', 0.9139779210090637),
 ('video', 0.9122210741043091),
 ('üretecek', 0.9101897478103638),
 ('eseri', 0.9087969064712524)]

In [96]:
model_with_nltk_token.wv.most_similar('erkek')

[('ödülüne', 0.9677895307540894),
 ('yapım', 0.9623268842697144),
 ('kuşak', 0.9611528515815735),
 ('eseri', 0.960191547870636),
 ('animasyon', 0.9574061036109924),
 ('imdat', 0.9562394618988037),
 ('tabanlı', 0.955939769744873),
 ('altındaki', 0.9548291563987732),
 ('sıhhi', 0.9544093608856201),
 ('telefonlar', 0.9516099691390991)]

In [91]:
vec = model_with_nltk_token.wv['kral']-model_with_nltk_token.wv['erkek']+model_with_nltk_token.wv['kadın']
model_with_nltk_token.wv.most_similar([vec])

[('kadın', 0.8680773377418518),
 ('grubu', 0.8530337810516357),
 ('özülker', 0.8443866968154907),
 ('kabiliyetleriyle', 0.8226409554481506),
 ('yarışması', 0.8165177702903748),
 ('müzik', 0.8094179630279541),
 ('gençlerle', 0.8063808083534241),
 ('mba', 0.8035210967063904),
 ('ticaret', 0.8019552230834961),
 ('eleme', 0.800213634967804)]

In [97]:
vec = model_with_nltk_token.wv['Almanya']-model_with_nltk_token.wv['Berlin']+model_with_nltk_token.wv['Paris']
model_with_nltk_token.wv.most_similar([vec])

KeyError: ignored

Apparently, text size is not enough to train an embedding